In [ ]:
from google.colab import drive
drive.mount('/drive/')

Mounted at /drive/


In [ ]:
!pip install yfinance

In [ ]:
!pip install mibian

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import zipfile, os
import yfinance as yf
import mibian
import time
import plotly
#import matplotlib
from pathlib import Path
from datetime import timedelta
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def loadcsvtodf(directory, filename):
  my_file = Path(directory+filename)
  if my_file.is_file():
    print('File {} found.'.format(filename))
    result = pd.read_csv(my_file)
    if result.empty:
      print('File {} empty.'.format(filename))
    else:
      print('File {} values loaded.'.format(filename))
  else:
    result = pd.DataFrame() 
    print('File {} NOT found !!!'.format(filename))
  return result

In [ ]:
def savetocsvwithwaiting(df, directory, filename, time_to_wait, mode): 
  # get saving time
  my_file = directory+filename
  header = True
  if Path(my_file).is_file():
    savetime_old = time.ctime(os.path.getmtime(my_file))
  else:
    savetime_old = '1900-01-01'
    mode = 'w'
  if mode == 'a':
    header = False  
  # save
  df.to_csv(my_file, mode=mode, header=header, index=False)
  # wait until saved
  time_counter = 0
  while not Path(my_file).is_file():
    time.sleep(1)
    time_counter += 1
    if time_counter > time_to_wait:break
  if Path(my_file).is_file():
    time_counter = 0
    while savetime_old == time.ctime(os.path.getmtime(my_file)):
      time.sleep(1)
      time_counter += 1
      if time_counter > time_to_wait:break

## basic parameters and loads

In [ ]:
working_directory = '/drive/MyDrive/OCM/data/SPX/'
toImport_directory = working_directory+'ToImport/'

tw_directory = '/drive/MyDrive/OCM/TW/SPX/'
tw_accounts = {'x2880': 'MB', 'x0197': 'MK'}

ideas_directory = '/drive/MyDrive/OCM/Ideas/SPX/'

tickerSymbol = "^SPX"
ticker = yf.Ticker(tickerSymbol)

In [ ]:
root = 'SPX'  # SPXW = weekly expiration 
option_type = 'C'

# RFR
interest_rate = 0.465  # in percentage - meaning 0.1% = 0.1

# lookback period
lookbackperiods = {'1W': 7, '1M': 30, '2M': 60}  # fix 3 pieces, otherwise function createdfiv has to be updated
minstrike = 2100
lookbackperiod_ideas = 60

# minimum days to expiration
mindte = 100

# minimum quote date
# minquotedate = '2018-01-01'

# miaximum quote date in base.csv file to keep the file size small
# maxquotedatebasecsv = '2020-10-01'

# delta range
mindelta =  0.01
maxdelta = 0.7

# minimum days history to show the strike
minstrikehistory = 15

# top open interest expirations
topexp = 5

# number of days on quote date slider
qdsliderdays = 20

# higher number will 'push' the ideas color faster
ideacoloraccelerator = 1  

# waiting time for csv's save
time_to_wait = 60

# PM margin
cost_contract = 0  #fix costs (probably 35) per contract, not sure whether exists that's why zero
# rule 1
# worst P/L case in the range
underlying_down = 0.2  #underlying asset down move e.g. 0.2 = 20% ove down scenario
underlying_up = 0.15  #underlying asset up move e.g. 0.15 = 15% move up scenario
# rule 2
naked_coef = 0.003  #default by broker 0.005, formula: naked_coef * underlying_price * naked_contracts * 100
# rule 3 - not used for now
vega_koef = 10 # vega_koef * net vega

# delta limits for inside sort / long / outsideshort
insideshortdeltafrom = 0.65
insideshortdeltato = 0.55
longdeltato = 0.35
outsideshortdeltafrom = 0.20
outsideshortdeltato = 0.10


In [ ]:
# colorscale for expiration cycle
bluered_colors, _ = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Bluered)
colorscale_bluered = plotly.colors.make_colorscale(bluered_colors)

## functions

In [ ]:
def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    if intermed <= 0 or len(colorscale) == 1:
        return colorscale[0][1]
    if intermed >= 1:
        return colorscale[-1][1]

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    # noinspection PyUnboundLocalVariable
    return plotly.colors.find_intermediate_color(
        lowcolor=low_color, highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb")

In [ ]:
def getexpmarks(dfhist_rank, quote_date):
  openinterestsum = dfhist_rank.query("quote_date == @quote_date")[['expiration','type','openinterest']].groupby(['expiration','type'],as_index=False).agg({"openinterest": 'sum'})
  openinterestsum = openinterestsum.sort_values('openinterest', ascending=False).head(topexp)
  result = np.sort(openinterestsum["expiration"])

  return result

In [ ]:
def addOwnGreeks(dfbase, source='CBOE', details=True):
  
  if source == 'CBOE':
    addstr='_1545'
    dfbase = dfbase[['quote_date','expiration','dte','strike','option_type','bid_1545','ask_1545','active_underlying_price_1545','delta_1545','vega_1545','implied_volatility_1545','trade_volume','open_interest']]
  else:
    addstr=''

  colname_underlaying_price = 'active_underlying_price'+addstr
  colname_bid = 'bid'+addstr  
  colname_ask = 'ask'+addstr
  
  # add clomn mid
  dfbase = dfbase.assign(mid = (dfbase[colname_bid]+dfbase[colname_ask])/2)

  # add own greeks...currently calls only !!!
  if details:
    dfbase['implied_volatility_calc'] = dfbase.apply(
        lambda row: mibian.BS([row[colname_underlaying_price], row['strike'], interest_rate, row['dte']], callPrice = row['mid']).impliedVolatility/100, axis = 1
        )
    dfbase['delta_calc'] = dfbase.apply(
        lambda row: mibian.BS([row[colname_underlaying_price], row['strike'], interest_rate, row['dte']], volatility = row['implied_volatility_calc']*100).callDelta, axis = 1
        )
    dfbase['vega_calc'] = dfbase.apply(
        lambda row: mibian.BS([row[colname_underlaying_price], row['strike'], interest_rate, row['dte']], volatility = row['implied_volatility_calc']*100).vega, axis = 1
        )
  
  return dfbase
  

In [ ]:
def calcRank(dfhist, valcurr, colname):
  min=dfhist[colname].min()
  max=dfhist[colname].max()
  if max>min:
    rank=100*(valcurr-min)/(max-min)
  else:
    rank=0 
  return rank

In [ ]:
def calcPercentile(dfhist, valcurr, colname):
  if colname =="implied_volatility_calc":
    nrlower = len(dfhist.query("implied_volatility_calc < @valcurr"))
  else:
    nrlower = 0 
  nrall=len(dfhist)
  if nrall>0:
    percentile = 100*nrlower/nrall
  else:
    percentile = 0
  return percentile

In [ ]:
def createdfiv(dfbase, dflookback, lookbackperiods=lookbackperiods, mindflookbackentries=minstrikehistory, owngreeks=True):
  
  if owngreeks:
    addstr='_calc'
  else:
    addstr='_1545'
  colname_iv = 'implied_volatility'+addstr
  colname_delta = 'delta'+addstr
  colname_vega = 'vega'+addstr

  #define dataframe columns
  df_iv = pd.DataFrame(columns=['quote_date', 'expiration', 'dte', 'underlying_price', 'strike','type','mid','iv','delta','vega','volume','openinterest'])
  df_iv = df_iv.reindex(columns=df_iv.columns.append('ivrank_'+pd.Index(lookbackperiods)))
  df_iv = df_iv.reindex(columns=df_iv.columns.append('ivpercentile_'+pd.Index(lookbackperiods)))
  
  #all expirations
  expirations = np.sort(dfbase["expiration"].unique())
  for expiration in expirations:
    df_ec = dfbase.query("expiration == @expiration")
    #all strikes
    strikes = np.sort(df_ec["strike"].unique())
    for strike in strikes:
      df_ec_strike = df_ec.query("strike == @strike")
      # all quote dates
      quote_dates = np.sort(df_ec_strike["quote_date"].unique())
      for quote_date in quote_dates:
        df_ec_strike_qd = df_ec_strike.query("quote_date == @quote_date")
        ivactual = df_ec_strike_qd[colname_iv].values[0]
        if 'active_underlying_price_1545' in df_ec_strike_qd:
          #CBOE source
          actual_underlying_price = df_ec_strike_qd['active_underlying_price_1545'].values[0]
          volume = df_ec_strike_qd['trade_volume'].values[0]
          openinterest = df_ec_strike_qd['open_interest'].values[0]
        else:
          actual_underlying_price = df_ec_strike_qd['active_underlying_price'].values[0]
          volume = df_ec_strike_qd['volume'].values[0]
          openinterest = df_ec_strike_qd['openInterest'].values[0]
        dfhelp=pd.DataFrame(columns=['ivrank', 'ivpercentile'])
        skipstrike=False
        for key, value in lookbackperiods.items():  #accessing keys and values
          lookback_date = datetime.strftime(datetime.strptime(quote_date, '%Y-%m-%d') -  pd.to_timedelta(value, unit='d'),'%Y-%m-%d')
          dflookback_ec_strike = dflookback.query("expiration == @expiration and strike == @strike and quote_date < @quote_date and quote_date >= @lookback_date")
          if value>=2*mindflookbackentries and len(dflookback_ec_strike)<mindflookbackentries:  #check min strike history requirements
            skipstrike=True
          ivrank = calcRank(dflookback_ec_strike,ivactual,colname_iv)
          ivpercentile = calcPercentile(dflookback_ec_strike,ivactual,colname_iv)
          dfhelp.loc[len(dfhelp.index)] = [ivrank, ivpercentile]
        
        if skipstrike==False:
          df_iv.loc[len(df_iv.index)] = [df_ec_strike_qd['quote_date'].values[0], df_ec_strike_qd['expiration'].values[0], df_ec_strike_qd['dte'].values[0],
                                         actual_underlying_price, strike, df_ec_strike_qd['option_type'].values[0], df_ec_strike_qd['mid'].values[0], ivactual,df_ec_strike_qd[colname_delta].values[0], 
                                         df_ec_strike_qd[colname_vega].values[0], volume, openinterest,
                                         dfhelp['ivrank'].values[0], dfhelp['ivrank'].values[1], dfhelp['ivrank'].values[2], 
                                         dfhelp['ivpercentile'].values[0], dfhelp['ivpercentile'].values[1], dfhelp['ivpercentile'].values[2]
                                         ]
                            
  return df_iv 

In [ ]:
def createdfiv_ec(dfbase, dflookback, colname_iv, lookbackperiods=lookbackperiods, mindflookbackentries=minstrikehistory):
  
  dflookback = pd.concat([dfbase,dflookback], ignore_index=True)

  #define dataframe columns
  df_iv = pd.DataFrame(columns=['quote_date', 'expiration', 'type', 'ivec_basis'])
  df_iv = df_iv.reindex(columns=df_iv.columns.append('ivrank_ec_'+pd.Index(lookbackperiods)))
  
  #all expirations
  expirations = np.sort(dfbase["expiration"].unique())
  for expiration in expirations:
    df_ec = dfbase.query("expiration == @expiration")
    # all quote dates
    quote_dates = np.sort(df_ec["quote_date"].unique())
    for quote_date in quote_dates:
      df_ec_qd = df_ec.query("quote_date == @quote_date")
      ivactual = df_ec_qd[colname_iv].values[0]
      dfhelp=pd.DataFrame(columns=['ivrank'])
      skipqd=False
      for key, value in lookbackperiods.items():  #accessing keys and values
        lookback_date = datetime.strftime(datetime.strptime(quote_date, '%Y-%m-%d') -  pd.to_timedelta(value, unit='d'),'%Y-%m-%d')
        dflookback_ec = dflookback.query("expiration == @expiration and quote_date < @quote_date and quote_date >= @lookback_date")
        if value>=2*mindflookbackentries and len(dflookback_ec)<mindflookbackentries:  #check min strike history requirements
          skipqd = True
        ivrank = calcRank(dflookback_ec,ivactual,colname_iv)
        dfhelp.loc[len(dfhelp.index)] = [ivrank]
        
      if skipqd==False:
        df_iv.loc[len(df_iv.index)] = [df_ec_qd['quote_date'].values[0], df_ec_qd['expiration'].values[0], df_ec_qd['type'].values[0], ivactual,
                                       dfhelp['ivrank'].values[0], dfhelp['ivrank'].values[1], dfhelp['ivrank'].values[2]]
                            
  return df_iv 

In [ ]:
def getcurrchain(expiration, option_type, quote_date):
  chain = pd.DataFrame()  #necessary???
  if option_type == 'C':
    chain = ticker.option_chain(expiration).calls[['strike','lastPrice','lastTradeDate','volume','bid','ask','openInterest','impliedVolatility']]
  else:
    if option_type == 'P':
      chain = ticker.option_chain(expiration).puts[['strike','lastPrice','lastTradeDate','volume','bid','ask','openInterest','impliedVolatility']]
  
  chain = chain.assign(option_type=option_type)
  chain = chain.assign(quote_date=quote_date)
  chain = chain.assign(expiration=expiration)
  
  # Underlaying asset additional info
  actual_underlying_price = ticker.history(period='1d')['Close'].values[0]
  chain = chain.assign(active_underlying_price=actual_underlying_price)

  return chain

In [ ]:
def getTradeStatusPL(dftw_ow_ec_qd, iv_ec):
  
  tw_status_price_open = dftw_ow_ec_qd['value_float'].sum()
  
  dftw_status_ow_ec_qd = dftw_ow_ec_qd.query("quantity_withsign != 0")
  tw_status = iv_ec.iloc[np.where(iv_ec.strike.isin(np.sort(dftw_status_ow_ec_qd["Strike_Price"].unique())))]
  tw_status = pd.merge(tw_status,dftw_status_ow_ec_qd, how='left',left_on=['strike'],right_on=['Strike_Price'])   
  tw_status_price_qd = (100*tw_status['mid']*tw_status['quantity_withsign']).sum()
  
  tw_status_PL_qd = tw_status_price_qd + tw_status_price_open
  
  return tw_status_PL_qd, tw_status

In [ ]:
def getIdeaStatusPL(owner, expiration, quote_date, lasttrade_close_date, dftwideas, iv_ec_current, dfhist_rank_ec):
  
  iv_ec = dfhist_rank_ec.query("quote_date == @quote_date")
  if iv_ec.empty:
    iv_ec = iv_ec_current
  dftw_ow_ec_qd = dftwideas.query("owner == @owner and expiration == @expiration and transaction_date <= @quote_date and transaction_date >= @lasttrade_close_date")[['owner','Strike_Price','trade_start_date','value_float','quantity_withsign']].groupby(['owner','Strike_Price','trade_start_date'],as_index=False).agg({"value_float": 'sum', "quantity_withsign": 'sum'})

  tw_status_PL_qd, tw_status = getTradeStatusPL(dftw_ow_ec_qd, iv_ec)  
  
  return tw_status_PL_qd

In [ ]:
def getRealizedStrikePL(owner, expiration, strike, action, quantity_withsign_realized, value_float, transaction_date, trade_start_date, dftw):
  action_search = ''
  if action == 'BUY_TO_CLOSE':
    action_search = 'SELL_TO_OPEN'
  else:
    if action == 'SELL_TO_CLOSE':
      action_search = 'BUY_TO_OPEN'
  
  dfsum = dftw.query("owner == @owner and expiration == @expiration and Strike_Price == @strike and Action == @action_search and transaction_date <= @transaction_date and transaction_date >= @trade_start_date")[['value_float','Quantity']].sum()
  if dfsum.Quantity !=0:
    avg_price = dfsum.value_float/dfsum.Quantity
    realizedStrikePL = (quantity_withsign_realized*avg_price).round(0)
    realizedStrikePL = realizedStrikePL + value_float
  else:
    realizedStrikePL = 0

  return realizedStrikePL


In [ ]:
def getexpirationprice(option_type, uderlying_price, strike):
  expiration_price = 0
  if option_type == 'C':
    if uderlying_price > strike:
      expiration_price = uderlying_price - strike
  if option_type == 'P':
    if uderlying_price < strike:
      expiration_price = strike - uderlying_price

  return expiration_price

In [ ]:
def getTradePL(underlying_price, tw_status, dftw_ow_ec_qd, exp=False):
  owner = tw_status['owner'].values[0]
  expiration = tw_status['expiration'].values[0]
  quote_date = tw_status['quote_date'].values[0]
  trade_start_date = tw_status['trade_start_date'].values[0]
  option_type = tw_status['type'].values[0]

  tw_status_price_open = dftw_ow_ec_qd['value_float'].sum()
  
  if exp:
    tw_status['mid_PL'] = tw_status.apply(
        lambda row: getexpirationprice(option_type, underlying_price, row["strike"]), axis=1
        )
  else:
    if option_type == 'C':
      tw_status['mid_PL'] = tw_status.apply(
          lambda row: mibian.BS([underlying_price, row['strike'], interest_rate, row['dte']], volatility = row['iv']*100).callPrice, axis = 1
          )
    else:
      if option_type == 'P':
        tw_status['mid_PL'] = tw_status.apply(
            lambda row: mibian.BS([underlying_price, row['strike'], interest_rate, row['dte']], volatility = row['iv']*100).putPrice, axis = 1
            )

  tw_price_qd = (100*tw_status['mid_PL']*tw_status['quantity_withsign']).sum()
  tw_PL_qd = tw_price_qd + tw_status_price_open
  
  return tw_PL_qd

In [ ]:
def getTradeGreek(underlying_price, tw_status, greek):

  if greek == 'vega':
    tw_status['vega_trade'] = tw_status.apply(
        lambda row: mibian.BS([underlying_price, row['strike'], interest_rate, row['dte']], volatility = row['iv']*100).vega, axis = 1
        )
  
  if greek == 'delta':
    if option_type == 'C':
      tw_status['delta_trade'] = tw_status.apply(
          lambda row: mibian.BS([underlying_price, row['strike'], interest_rate, row['dte']], volatility = row['iv']*100).callDelta, axis = 1
          )
    else:
      if option_type == 'P':
        tw_status['delta_trade'] = tw_status.apply(
            lambda row: mibian.BS([underlying_price, row['strike'], interest_rate, row['dte']], volatility = row['iv']*100).putDelta, axis = 1
            )     

  tw_greek = (100*tw_status[greek+'_trade']*tw_status['quantity_withsign']).sum()
  
  return tw_greek

In [ ]:
def getPortfolioMarginRequirement(underlying_price, iv_ec, naked_contracts, total_contracts):
  # min of 2 rules

  # rule 1 ... with strike simplification which doesn't hurt at all...
  strike_min = iv_ec['strike'].min()
  strike_min = max(strike_min, underlying_price * (1-underlying_down))
  strike_down = iv_ec.query("strike <= @strike_min")['strike'].max()
  strike_max = iv_ec['strike'].max()
  strike_max = min(strike_max, underlying_price * (1+underlying_up))
  strike_up = iv_ec.query("strike >= @strike_max")['strike'].min()
  margin_r1 = iv_ec.query("strike >= @strike_down and strike <= @strike_up")['pl_theoretical'].min()
  margin_r1 = margin_r1 - total_contracts * cost_contract
  # rule 2
  margin_r2 = naked_coef * underlying_price * naked_contracts * 100  - total_contracts * cost_contract

  margin = abs(min(margin_r1, margin_r2))
  
  return margin


In [ ]:
def prepareTrade(dftw, tw=True):
  dftw = dftw.assign(quantity_withsign = dftw['Quantity'])
  dftw.loc[(dftw["Action"].str.contains("SELL")), 'quantity_withsign'] = -1*dftw["Quantity"]
  dftw = dftw.assign(quantity_withsign_realized = dftw['quantity_withsign'],)
  dftw.loc[(dftw["Action"].str.contains("OPEN")), 'quantity_withsign_realized'] = 0 
  dftw = dftw.assign(value_float = dftw['Value'])
  if tw:
    dftw["value_float"] = dftw["value_float"].str.replace(',','')
    dftw["value_float"] = dftw["value_float"].astype('float')
    dftw = dftw.assign(expiration = pd.to_datetime(dftw["Expiration_Date"]).dt.date.astype('str'))
    dftw = dftw.assign(transaction_date = pd.to_datetime(dftw["Date"].str[0:10]).dt.date.astype('str'))
    # set the minimum transaction date as the default for trade start 
    dftw = dftw.assign(trade_start_date = dftw['transaction_date'])
    dftw_exp_mintransactiondate = dftw[['owner','expiration','transaction_date']].groupby(['owner','expiration'],as_index=False).agg({"transaction_date": 'min'})
    for i in dftw_exp_mintransactiondate.itertuples():
      dftw.loc[(dftw['owner']==i.owner) & (dftw['expiration']==i.expiration), 'trade_start_date'] = i.transaction_date
    # update trade start date
    dftw_tradeclosed = pd.DataFrame(columns=['owner', 'expiration', 'trade_close_date'])
    dftw_ow_exp_strike_td = dftw[['owner','expiration','Strike_Price','transaction_date','quantity_withsign']].groupby(['owner','expiration','Strike_Price','transaction_date'],as_index=False).agg({"quantity_withsign": 'sum'})
    for i in dftw_ow_exp_strike_td.itertuples():
      if len(dftw_tradeclosed.query("trade_close_date == @i.transaction_date")) == 0:
        dftw_quantity = dftw_ow_exp_strike_td.query("owner == @i.owner and expiration == @i.expiration and transaction_date <= @i.transaction_date").groupby(['owner','expiration','Strike_Price'], as_index=False).agg({"quantity_withsign": 'sum'}).query("quantity_withsign !=0")
        if len(dftw_quantity) == 0:
          dftw_tradeclosed.loc[len(dftw_tradeclosed.index)] = [i.owner,i.expiration,i.transaction_date]
    for i in dftw_tradeclosed.itertuples():
      nexttrade_start_date = dftw.query("owner == @i.owner and expiration == @i.expiration and transaction_date > @i.trade_close_date")['transaction_date'].min()
      dftw.loc[(dftw['owner']==i.owner) & (dftw['expiration']==i.expiration) & (dftw['transaction_date']>=nexttrade_start_date), 'trade_start_date'] = nexttrade_start_date
  
  if tw:
    return dftw, dftw_tradeclosed
  else:
    return dftw

In [ ]:
def getStrikeMid(strike, dfsource):
  mid = dfsource.query("strike == @strike")['mid'].values[0]
  return mid

In [ ]:
def adaptIdeas(dfideas, dftw):
  owner = dfideas["owner"].to_numpy()
  active = dfideas["active"].to_numpy()
  quantity = dfideas["quantity"].to_numpy()
  strike = dfideas["Strike_Price"].to_numpy()
  action = dfideas["Action"].to_numpy()
  price = dfideas["price"].to_numpy()

  dfideasadapted = pd.DataFrame(columns=['owner', 'active', 'quantity', 'Strike_Price', 'Action', 'price'])

  for i, s in enumerate(strike):
    q = quantity[i]
    quantity_open = dftw.query('Strike_Price == @s')['quantity_withsign'].sum()
    if action[i] == 'BUY':
      dftw.loc[len(dftw.index)] = [s, q]
      quantity_to_close = max(0, q - max(0, quantity_open + q))
      if quantity_to_close > 0:
        dfideasadapted.loc[len(dfideasadapted.index)] = [owner[i], active[i], quantity_to_close, s, 'BUY_TO_CLOSE', price[i]]
      quantity_to_open = q - quantity_to_close
      if quantity_to_open > 0:
        dfideasadapted.loc[len(dfideasadapted.index)] = [owner[i], active[i], quantity_to_open, s, 'BUY_TO_OPEN', price[i]]
    else:
      dftw.loc[len(dftw.index)] = [s, (-1)*q]
      quantity_to_close = max(0, q + min(0, quantity_open - q))
      if quantity_to_close > 0:
        dfideasadapted.loc[len(dfideasadapted.index)] = [owner[i], active[i], quantity_to_close, s, 'SELL_TO_CLOSE', price[i]]
      quantity_to_open = q - quantity_to_close
      if quantity_to_open > 0:
        dfideasadapted.loc[len(dfideasadapted.index)] = [owner[i], active[i], quantity_to_open, s, 'SELL_TO_OPEN', price[i]]

  return dfideasadapted

In [ ]:
def prepareTradeFromIdeas(dfideas, quote_date, expiration, trade_start_date, iv_ec, ideatype):
  dfideas = dfideas.query("active == True and quantity>0 and Strike_Price>0")
  now = str(datetime.now().isoformat())
  dfideas = dfideas.assign(Date=now, Type=ideatype, Symbol='SPX', Instrument_Type='Equity Option', Description='NA', 
                           Commission=0.0, Fees=0.0, Multiplier=100, Underlying_Symbol='SPX', Expiration_Date='NA',
                           Call_or_Put='CALL', Order_nr=100000000, transaction_date=quote_date, expiration=expiration, trade_start_date=trade_start_date,
                           )
  
  dfideas["Quantity"] = dfideas["quantity"].astype('int') 
  
  #Set Price
  dfideas['Average_Price'] = dfideas.apply(
    lambda row: 100*row["price"] if row["price"]>0 else 100*getStrikeMid(row["Strike_Price"], iv_ec), axis=1
    )
  dfideas.loc[(dfideas["Action"].str.contains("BUY")), 'Average_Price'] = -1*dfideas["Average_Price"]

  dfideas = dfideas.assign(Value=dfideas['Average_Price']*dfideas['Quantity'])
  dfideas = dfideas.assign(value_float=dfideas['Value'])

  dfideas = prepareTrade(dfideas, tw=False)

  dfideas = dfideas.drop(['quantity', 'active', 'price'], axis = 1) 
  
  return dfideas

## import new data from ToImport directory

In [ ]:
# import new data from ToImport directory
regenerate = False #set True if dfhist and dfhist_rank should be regenerated
newimport = False
dfnewimport=pd.DataFrame()
os.chdir(toImport_directory)
for file in os.listdir(toImport_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile
        with zipfile.ZipFile(file, 'r') as item:  # create Zipfile object and name it item
          #item.extractall()  # extract it in the working directory
          itemlist=item.namelist()
          for name in itemlist:
            with item.open(name) as csvfile:
              dfnewimport = pd.concat([dfnewimport,pd.read_csv(csvfile)], ignore_index=True) # all new imported csv's
              print('New import: file {}.'.format(csvfile.name))
              newimport = True 
          os.remove(toImport_directory+file)
          print('File {} deleted from ToImport.'.format(file))    

if newimport or regenerate:
  # pre-prepare New import data
  dfnewimport = dfnewimport.query("root == @root and option_type == @option_type and strike >= @minstrike and strike%50 == 0 and bid_1545 > 0 and ask_1545 > 0")
  dfnewimport = dfnewimport.assign(dte = (pd.to_datetime(dfnewimport['expiration']).astype('datetime64[ns]')-pd.to_datetime(dfnewimport['quote_date']).astype('datetime64[ns]')).dt.days)
  # add new imports to dfhist
  dfhistnewimport = dfnewimport.query("dte > @mindte")    # and quote_date >= @minquotedate
  dfhistnewimport = addOwnGreeks(dfbase=dfhistnewimport)
  if regenerate:
    dfhist = dfhistnewimport
  else:
    dfhist = loadcsvtodf(working_directory,'dfhist.csv') # filtered columns + own greeks
    dfhist = pd.concat([dfhist,dfhistnewimport], ignore_index=True)
  savetocsvwithwaiting(dfhist, working_directory, 'dfhist.csv', time_to_wait, 'w')
  print('New imports added to dfhist.csv'+'...SAVED')
  # add new imports to dfhist_rank
  dfhist_ranknewimport = createdfiv(dfhistnewimport, dfhist)
  if regenerate:
    dfhist_rank = dfhist_ranknewimport
  else:
    dfhist_rank = loadcsvtodf(working_directory,'dfhist_rank.csv') # dfhist + ranks + percentiles
    dfhist_rank = pd.concat([dfhist_rank,dfhist_ranknewimport], ignore_index=True)
  savetocsvwithwaiting(dfhist_rank, working_directory, 'dfhist_rank.csv', time_to_wait, 'w')
  print('New imports added to dfhist_rank.csv'+'...SAVED')
  # add new imports to dfhist_ec_rank
  dfhist_ec_ranknewimport = dfhist_ranknewimport[['quote_date','expiration','type','openinterest']].assign(ivec_basis = dfhist_ranknewimport['openinterest']*dfhist_ranknewimport['iv'])
  dfhist_ec_ranknewimport = dfhist_ec_ranknewimport[['quote_date','expiration','type','openinterest','ivec_basis']].groupby(['quote_date','expiration','type'],as_index=False).agg({"openinterest": 'sum',"ivec_basis": 'sum'})
  dfhist_ec_ranknewimport['ivec_basis'] = dfhist_ec_ranknewimport['ivec_basis']/dfhist_ec_ranknewimport['openinterest']
  if regenerate:
    dfhist_ec_rank = dfhist_ec_ranknewimport
  else:
    dfhist_ec_rank = loadcsvtodf(working_directory,'dfhist_ec_rank.csv') # history of expiration cycles IVR
  dfhist_ec_ranknewimport = createdfiv_ec(dfhist_ec_ranknewimport, dfhist_ec_rank, 'ivec_basis')
  if regenerate:  # yes, once again
    dfhist_ec_rank = dfhist_ec_ranknewimport
  else:
    dfhist_ec_rank = pd.concat([dfhist_ec_rank,dfhist_ec_ranknewimport], ignore_index=True)
  savetocsvwithwaiting(dfhist_ec_rank, working_directory, 'dfhist_ec_rank.csv', time_to_wait, 'w')
  print('New imports added to dfhist_ec_rank.csv'+'...SAVED')
else:
  # load csv's
  dfhist = loadcsvtodf(working_directory,'dfhist.csv') # filtered columns + own greeks
  dfhist_rank = loadcsvtodf(working_directory,'dfhist_rank.csv') # dfhist + ranks + percentiles
  dfhist_ec_rank = loadcsvtodf(working_directory,'dfhist_ec_rank.csv') # history of expiration cycles IVR

#time measure for if performance fine-tunning necessary
# t = time.process_time()
# #action
# elapsed_time = time.process_time() - t
# print(elapsed_time)

In [ ]:
dfhist_ec_rank.head()

In [ ]:
# if new import to dfhist_rank crashed

# dfhist = loadcsvtodf(working_directory,'dfhist.csv')
# dfhistnewimport = dfhist.query("quote_date>'2000-01-01'") # define the date
# dfhist_ranknewimport = createdfiv(dfhistnewimport, dfhist)
# # dfhist_rank = pd.DataFrame() # if dfhist_rank should be completely re-generated
# dfhist_rank = pd.concat([dfhist_rank,dfhist_ranknewimport], ignore_index=True)
# savetocsvwithwaiting(dfhist_rank, working_directory, 'dfhist_rank.csv', time_to_wait, 'w')
# print('New imports added to dfhist_rank.csv'+'...SAVED')

# dfhist_ranknewimport = dfhist_rank.query("quote_date>'2000-01-01'") # define the date
# dfhist_ec_rank = pd.DataFrame() # if dfhist_ec_rank should be completely re-generated
# dfhist_ec_ranknewimport = dfhist_ranknewimport[['quote_date','expiration','type','openinterest']].assign(ivec_basis = dfhist_ranknewimport['openinterest']*dfhist_ranknewimport['iv'])
# dfhist_ec_ranknewimport = dfhist_ec_ranknewimport[['quote_date','expiration','type','openinterest','ivec_basis']].groupby(['quote_date','expiration','type'],as_index=False).agg({"openinterest": 'sum',"ivec_basis": 'sum'})
# dfhist_ec_ranknewimport['ivec_basis'] = dfhist_ec_ranknewimport['ivec_basis']/dfhist_ec_ranknewimport['openinterest']
# dfhist_ec_ranknewimport = createdfiv_ec(dfhist_ec_ranknewimport, dfhist_ec_ranknewimport, 'ivec_basis', lookbackperiods)
# dfhist_ec_rank = pd.concat([dfhist_ec_rank,dfhist_ec_ranknewimport], ignore_index=True)
# savetocsvwithwaiting(dfhist_ec_rank, working_directory, 'dfhist_ec_rank.csv', time_to_wait, 'w')
# print('New imports added to dfhist_ec_rank.csv'+'...SAVED')

## set sliders

In [ ]:
qdmarks = np.sort(dfhist_rank["quote_date"].unique())
# add now date if doesn't exist
now = datetime.today().strftime('%Y-%m-%d')
if now not in qdmarks:
  qdmarks = np.append(qdmarks, now)

lastqd = dfhist_rank['quote_date'].max()
expmarks = getexpmarks(dfhist_rank, lastqd)

## import real trades

In [ ]:
dftw = pd.DataFrame()
os.chdir(tw_directory)
for file in os.listdir(tw_directory):
  if file.endswith(".csv"):
    dfaccount = pd.read_csv(file)
    dfaccount.columns = dfaccount.columns.str.replace(' #','_nr')
    dfaccount.columns = dfaccount.columns.str.replace(' ','_')
    owner = 'unknown'
    for key, value in tw_accounts.items():
      if key in file:
        owner = value
    dfaccount = dfaccount.assign(owner = owner)
    dftw = pd.concat([dftw,dfaccount], ignore_index=True)
    print('Trades imported from file {}.'.format(file))

In [ ]:
dftw, dftw_tradeclosed = prepareTrade(dftw, tw=True)

In [ ]:
# delete old user interface trades, remove ideas files
trades_filename = 'trades.csv'
dftrades = loadcsvtodf(ideas_directory,trades_filename) 
if not dftrades.empty:
  owners = dftw["owner"].unique()
  for owner in owners:
    max_transaction_date = dftw.query("owner == @owner")['transaction_date'].max()
    dftrades = dftrades.query("transaction_date > @max_transaction_date")
  savetocsvwithwaiting(dftrades, ideas_directory, trades_filename, time_to_wait, 'w')

##Chart Dash

In [ ]:
!pip install jupyter-dash

In [ ]:
!pip install dash-bootstrap-components

In [ ]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State, MATCH, ALL
import plotly.graph_objects as go
#import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
def SetColorTWStatus(x):
  if x >= 0:
    return "green"
  else:
    return "red"

In [ ]:
empty_row = dbc.Label(html.P(""))

qd_calendar = dcc.DatePickerSingle(
    id='calendar-qd',
    min_date_allowed=qdmarks[0],
    max_date_allowed=qdmarks[len(qdmarks)-1],
    date=qdmarks[len(qdmarks)-2],
    display_format='MMM D, YYYY',
    # number_of_months_shown = 3 
)

qd_slider = dbc.FormGroup(
    [
        # dbc.Label("Quote Date"),
        dcc.Slider(
                  id='slider-qd',
                  marks={int(i):j[-2:]+'.'+j[5:7] for i,j in zip(range(len(qdmarks)), qdmarks)},
                  min=len(qdmarks)-qdsliderdays,
                  max=len(qdmarks)-1,
                  value=len(qdmarks)-2,
                  step=None,
                  updatemode='drag',
                  included=False,
                  vertical=True,
                  verticalHeight=450,
        )
    ]
)

exp_dropdown = dcc.Dropdown(
    id='dropdown-exp',
    # options=[{'label':exp[5:7]+'/'+exp[:4], 'value':exp} for exp in expmarks],
    options=[{'label':exp, 'value':exp} for exp in expmarks],
    value = expmarks[-1],
    disabled=True,
    clearable=False,
    style={
    # 'min-width': '120px', 
    'font-size': "110%",
    'min-height': '40px',
    },
    )

exp_slider = dbc.FormGroup(
    [
        # dbc.Label("Expiration"),
        dcc.Slider(
                  id='slider-exp',
                  marks={int(i):j[5:7]+'/'+j[:4] for i,j in zip(range(len(expmarks)), expmarks)},
                  min=0,
                  max=len(expmarks)-1,
                  value=len(expmarks)-1,
                  step=None,
                  updatemode='drag',
                  included=False,
                  vertical=True,
                  verticalHeight=400
        )
    ]
)

iv_checklist = dbc.Checklist(
    id='checklist-iv',
    options=[
        {'label': 'Open Interest', 'value': 'OI'},
        {'label': 'Volume', 'value': 'VOL'},
        {'label': 'IV Percentile', 'value': 'IVP'},
    ],
    value=['OI'],
    inline=True,
)  

rank_checklist = dbc.Checklist(
    id='checklist-rank',
    options=[
        {'label': '1W', 'value': '1W'},
        {'label': '1M', 'value': '1M'},
        {'label': '2M', 'value': '2M'},
    ],
    value=['1M'],
    inline=True,
)

dayaction_radioitems = dbc.FormGroup(
    [
     dbc.Label("Day Action:", html_for="example-radios-row"),
     dbc.Col(
         dbc.RadioItems(
             id='radioitems-dayaction',
             options=[
                      {'label': 'None', 'value': 'none'},
                      {'label': 'Realized', 'value': 'realized'},
                      {'label': 'All', 'value': 'all'},
                      ],
             value='none',
             inline=True,
         ),
     ),
    ],
    row=True,
)

add_button = dbc.Button(
    "Add Row",
    id="add-idea",
    n_clicks=0,
    outline=True,
    size="md",
    color="primary"
)

project_button = dbc.Button(
    "Project Selection",
    id="project-ideas",
    n_clicks=0,
    # outline=True,
    size="md",
    color="secondary"
)

apply_button = dbc.Button(
    "Apply Selection",
    id="apply-ideas",
    n_clicks=0,
    # outline=True,
    size="md",
    color="success"
)

# apply_button = html.Div([
#     dcc.ConfirmDialog(
#         id='confirm-apply',
#         message='Confirm Selection --> Trade',
#     ),
#     dbc.Button(
#         "Apply Selection",
#         id="apply-ideas",
#         n_clicks=0,
#         # outline=True,
#         size="md",
#         color="success"
#     )
# ])

clear_button = dbc.Button(
    "Clear All",
    id="clear-all",
    n_clicks=0,
    outline=True,
    size="sm",
    color="danger"
)

trades_info_buttons = html.Div(
    [
     dbc.Button(
         "Trades List",
         id="show-tradeslist",
         n_clicks=0,
        #  outline=True,
         size="sm",
         color="info"
      ),
      dbc.Popover(
          [
          # dbc.PopoverHeader("Trades List"),
          dbc.PopoverBody(id="table-tradeslist"),
          ],
          id='popover-tradeslist',
          target="show-tradeslist",
          is_open=False,
          placement='right',
      ),
     dbc.Button(
         "Trade action dates",
         id="show-tradeactiondates",
         n_clicks=0,
        #  outline=True,
         size="sm",
         color="info"
      ),
      dbc.Popover(
          [
          # dbc.PopoverHeader("Action Dates"),
          dbc.PopoverBody(id="table-tradeactiondates"),
          ],
          id='popover-tradeactiondates',
          target="show-tradeactiondates",
          is_open=False,
          placement='right',
      ),
     dbc.Button(
         "Day Actions",
         id="show-dayactions",
         n_clicks=0,
        #  outline=True,
         size="sm",
         color="info",
        #  className="mb-3",
      ),
      dbc.Collapse(
          dbc.Card(dbc.CardBody(id="table-dayactions")),
          id='collapse-dayactions',
          is_open=False,
      ),
    ]
)

ideas_graph = dbc.Spinner(
    id="loading-graph-ideas",
    color="secondary",
    children=[
              html.Div(dcc.Graph(
                  id='graph-ideas',
                  config={
                      'displayModeBar': False,
                      'displaylogo': False,
                      # 'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                      # 'modeBarButtonsToAdd':['drawline','drawopenpath','drawclosedpath','drawcircle','drawrect','eraseshape']
                      },
                  )
              )]
)

owner_card = dbc.Card(
    [
        dbc.CardHeader(
            dbc.Tabs(
                [
                    dbc.Tab(label="Mikes", tab_id="MB"),
                    dbc.Tab(label="Sujdo", tab_id="MK"),
                    dbc.Tab(label="Paper", tab_id="PT"),
                ],
                id="tabs-owner",
                card=True,
                active_tab="MB",
            )
        ),
        dbc.CardBody([
                      trades_info_buttons, 
                      html.Div(empty_row),
                      clear_button,
                      html.Div(id="idea-container", children=[]),
                      # html.Div(id='check-output', children=[]),
                      add_button,
                      html.Div(empty_row),
                      html.Div([project_button, apply_button]),
        ]),
    ],
    # style={"width": "25rem"},
)

iv_graph = dbc.Spinner(
    # id="loading-graph-iv",
    color="primary",
    children=[
              html.Div(dcc.Graph(
                  id='graph-iv',
                  config={
                      'displayModeBar': False,
                      'displaylogo': False,
                      # 'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                      # 'modeBarButtonsToAdd':['drawline','drawopenpath','drawclosedpath','drawcircle','drawrect','eraseshape']
                      },
                  )
              ),
              html.Div(id="table-tradeStatusOverview")
              ]
)

strike_input = dcc.Input(
    id={
        'type': 'input-strike',
        'index': 'n_clicks'
        },
    type="number", 
    min=2500, 
    max=7000, 
    step=50, 
    placeholder = 'Strike',
    required=True,
)

price_input = dcc.Input(
    id={
        'type': 'input-price',
        'index': 'n_clicks'
        },
    type="number", 
    min=0, 
    max=3000, 
    step=0.01, 
    placeholder = 'Price',
)

quantity_input = dcc.Input(
    id={
        'type': 'input-quantity',
        'index': 'n_clicks'
        },
    type="number", 
    min=1, 
    max=1000, 
    step=1, 
    placeholder = 'Quantity',
    required=True,
)


action_dropdown = dcc.Dropdown(
    id={
        'type': 'dropdown-action',
        'index': 'n_clicks'
        },
    options=[{'label': i, 'value': i} for i in ['BUY', 'SELL']],
    # placeholder="Act",
    value='SELL',
    clearable=False,
    searchable=True,
    style={
    # 'height': '2px', 
    'width': '65px', 
    # 'font-size': "50%",
    'min-height': '1px',
    },
)

active_checklist = dcc.Checklist(
    id={
        'type': 'checklist-active',
        'index': 'n_clicks'
        },
    options=[{'label': '    ' , 'value': 'T'}],
    value=['T'],         
)

remove_button = html.Button(
    'Remove',
    id={
        'type': 'button-remove',
        'index': 'n_clicks'
        },
)

#fig.show(config={"displayModeBar": True, "showTips": False})  #if not shown via Dash, otherwise "app"

#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/cyborg/bootstrap.min.css']
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.config.suppress_callback_exceptions = True


app.layout = dbc.Container(
    [
     dbc.Row([dbc.Label(html.H5("SPX Calls"), width=2),
              ]),
     dbc.Row([dbc.Col(exp_dropdown, width=1),
              dbc.Col(qd_calendar, width="auto"),   # width={"size": 'auto', "offset": 1}
              dbc.Col(rank_checklist, width="auto"), 
              dbc.Col(iv_checklist, width={"size": 'auto', "offset": 1}),
              dbc.Col(dayaction_radioitems, width={"size": 'auto', "offset": 1})
              ]),
     dbc.Row([
              dbc.Col(exp_slider, width=1),
              dbc.Col(qd_slider, width=1),
              dbc.Col(iv_graph),
              ], align='start', justify='center'),
     dbc.Row([
              dbc.Col(owner_card, width=4),
              dbc.Col(ideas_graph),
              ], align='start', justify='center'),
    ],
    className="p-2",
    fluid=True,
)

#--------------Trades List-------------------
@app.callback(
Output('popover-tradeslist', 'is_open'),
Input('show-tradeslist', 'n_clicks'),
State('popover-tradeslist', 'is_open'),
)
def tradeslist_popover(n, is_open):
  # if n:
  #   isopen =  not is_open
  triggered = [t["prop_id"] for t in dash.callback_context.triggered]
  if 'show-tradeslist' in triggered[0]:
    isopen =  not is_open

  return isopen

#--------------Trade Action Dates-------------------
@app.callback(
Output('popover-tradeactiondates', 'is_open'),
Input('show-tradeactiondates', 'n_clicks'),
State('popover-tradeactiondates', 'is_open'),
)
def tradeactiondates_popover(n, is_open):
  # if n:
  #   isopen =  not is_open
  triggered = [t["prop_id"] for t in dash.callback_context.triggered]
  if 'show-tradeactiondates' in triggered[0]:
    isopen =  not is_open

  return isopen

#--------------Day Actions Collapse-------------------
@app.callback(
    Output("collapse-dayactions", "is_open"),
    Input("show-dayactions", "n_clicks"),
    State("collapse-dayactions", "is_open"),
)
def dayactions_collapse(n, is_open):
  # if n:
  #   isopen =  not is_open
  triggered = [t["prop_id"] for t in dash.callback_context.triggered]
  if 'show-dayactions' in triggered[0]:
    isopen =  not is_open

  return isopen

# # if confirmdialog is used
# @app.callback(
#     Output('confirm-apply', 'displayed'),
#     Input('apply-ideas', 'n_clicks'),
# )
# def display_confirm_apply(n):
#   triggered = [t["prop_id"] for t in dash.callback_context.triggered]
#   if 'apply-ideas' in triggered[0]:
#         return True
#   return False
#--------------ideas-------------------
@app.callback(
    Output('idea-container', 'children'),
    Input('tabs-owner', 'active_tab'),
    Input('project-ideas', 'n_clicks'),
    Input('apply-ideas', 'n_clicks'),
    # Input('confirm-apply', 'submit_n_clicks'),  # if confirmdialog is used...also change 'apply-ideas' in triggered[0] -> 'confirm-apply' in triggered[0]
    Input('clear-all', 'n_clicks'),
    Input('add-idea', 'n_clicks'),
    Input({'type': 'checklist-active', 'index': ALL}, 'value'),
    Input({'type': 'input-quantity', 'index': ALL}, 'value'),
    Input({'type': 'input-strike', 'index': ALL}, 'value'),
    Input({'type': 'dropdown-action', 'index': ALL}, 'value'),
    Input({'type': 'input-price', 'index': ALL}, 'value'),
    Input({'type': 'button-remove', 'index': ALL}, 'n_clicks'),
    State('idea-container', 'children'),
    )
def update_ideas(owner, project, apply, clear, add, active, quantity, strike, action, price, remove, ideas):  

  # if idea-container:
  #   raise dash.exceptions.PreventUpdate
  new_dropdown = dbc.FormGroup([active_checklist, quantity_input, strike_input, action_dropdown, price_input, remove_button], row=True)
  
  triggered = [t["prop_id"] for t in dash.callback_context.triggered]
  if 'add-idea' in triggered[0]:
    ideas.append(new_dropdown)
  if 'button-remove' in triggered[0]:
    for (i, value) in enumerate(remove):
      if value==1:
        del ideas[i]
        break
  if 'clear-all' in triggered[0]:
    ideas.clear()

  ideascopy = ideas
  ii = 0
  if 'project-ideas' in triggered[0] or 'apply-ideas' in triggered[0]:
    dfideas = pd.DataFrame(columns=['owner', 'active', 'quantity', 'Strike_Price', 'Action', 'price'])
    for (i, value) in enumerate(active):
      if value == ['T']:
        flagactive=True
      else:
        flagactive=False
      dfideas.loc[len(dfideas.index)] = [owner, flagactive, quantity[i], strike[i], action[i], price[i]]
      if 'apply-ideas' in triggered[0] and flagactive:
        del ideascopy[ii]
      else:
        ii = ii +1
    ideas_file = 'ideas_'+owner+'.csv'
    savetocsvwithwaiting(dfideas, ideas_directory, ideas_file, time_to_wait, 'w')
  
  return ideascopy

#--------------graphs-------------------
@app.callback(
    Output('table-tradeactiondates', 'children'),
    Output('table-tradeslist', 'children'),
    Output('table-dayactions', 'children'),
    Output('calendar-qd', 'date'),
    Output('slider-qd', 'value'),
    Output('dropdown-exp', 'options'),
    Output('dropdown-exp', 'value'),
    Output('slider-exp', 'value'),
    Output('slider-exp', 'marks'),
    Output('table-tradeStatusOverview', 'children'),
    Output('graph-ideas', 'figure'),
    Output('graph-iv', 'figure'),
    Input('project-ideas', 'n_clicks'),
    Input('apply-ideas', 'n_clicks'),
    # Input('confirm-apply', 'submit_n_clicks'),   # if confirmdialog is used...also change 'apply-ideas' in triggered[0] -> 'confirm-apply' in triggered[0]
    Input('calendar-qd', 'date'),
    Input('slider-qd', 'value'),
    Input('slider-exp', 'value'),
    Input('checklist-iv', 'value'),
    Input('checklist-rank', 'value'),
    Input('radioitems-dayaction', 'value'),
    Input('tabs-owner', 'active_tab'),
    Input('idea-container', 'children'),
    State('dropdown-exp', 'value'),
    )
def update_figure(project, apply, qdcal, qdseq, expseq, checklistiv, checklistrank, dayaction, owner, ideas, expdrop):
  triggered = [t["prop_id"] for t in dash.callback_context.triggered]
  
  if not 'slider-qd' in triggered[0]:
    qdseq = np.where(qdmarks >= qdcal)[0][0]
  qdseqreturn = max(qdseq, len(qdmarks)-qdsliderdays) # to show the lowest day of slider if the date from calendar is out of the bounds
  
  quote_date =  qdmarks[qdseq]
  # Dig the dates for expiration slider
  if qdseq == len(qdmarks)-1:  # for today we don't have information about all expirations
    expmarks = getexpmarks(dfhist_rank, qdmarks[qdseq-1])
  else:
    expmarks = getexpmarks(dfhist_rank, quote_date)
  expslidermarks = {int(i):j[5:7]+'/'+j[:4] for i,j in zip(range(len(expmarks)), expmarks)}
  expdropopt = [{'label':exp, 'value':exp} for exp in expmarks] # [{'label':exp[5:7]+'/'+exp[:4], 'value':exp} for exp in expmarks]
 
  if 'slider-exp' in triggered[0]:
    expiration = expmarks[expseq]
  else: 
    expiration =  expdrop

  if expiration in expmarks:
    expseqreturn = np.where(expmarks == expiration)[0][0]
  else:
    expseqreturn = len(expmarks)-1
    expiration = expmarks[-1]

  iv_ec = dfhist_rank.query("expiration == @expiration and quote_date == @quote_date")
  iv_ec_color = dfhist_ec_rank.query("expiration == @expiration and quote_date == @quote_date")
  if iv_ec.empty:
     # take data from yfinance
    dfactual = getcurrchain(expiration, option_type, quote_date)
    dfactual = dfactual.assign(dte = (pd.to_datetime(dfactual['expiration']).astype('datetime64[ns]')-pd.to_datetime(dfactual['quote_date']).astype('datetime64[ns]')).dt.days)
    dfactual = dfactual.query("option_type == @option_type and strike >= @minstrike and strike%50 == 0 and bid > 0.0 and ask > 0.0 and dte > @mindte")
    dfactual = addOwnGreeks(dfbase=dfactual, source='yahoofinance')
    iv_ec = createdfiv(dfactual, dfhist)
    dfactual_ec = iv_ec[['quote_date','expiration','type','openinterest']].assign(ivec_basis = iv_ec['openinterest']*iv_ec['iv'])
    dfactual_ec = dfactual_ec[['quote_date','expiration','type','openinterest','ivec_basis']].groupby(['quote_date','expiration','type'],as_index=False).agg({"openinterest": 'sum',"ivec_basis": 'sum'})
    dfactual_ec['ivec_basis'] = dfactual_ec['ivec_basis']/dfactual_ec['openinterest']
    iv_ec_color = createdfiv_ec(dfactual_ec, dfhist_ec_rank, 'ivec_basis')
    
  iv_ec = iv_ec.query("delta >= @mindelta and delta <= @maxdelta")

  underlying_price = iv_ec['underlying_price'].values[0]
  dte = iv_ec['dte'].values[0]
    
  lasttrade_close_date = dftw_tradeclosed.query("owner == @owner and expiration == @expiration and trade_close_date <= @quote_date")['trade_close_date'].max()
  if pd.isnull(lasttrade_close_date):
    lasttrade_close_date = '1900-01-01'

  if not dftw.empty:
    dftwideas_allexp = dftw.query("owner == @owner")
    dftwideas = dftw.query("owner == @owner and expiration == @expiration")
  
  filenameTrades = 'trades.csv'
  dftrades = loadcsvtodf(ideas_directory, filenameTrades)
  if not dftrades.empty:
    dftrades_allexp = dftw.query("owner == @owner")
    if not dftrades_allexp.empty:
      dftwideas_allexp = pd.concat([dftwideas_allexp,dftrades_allexp], ignore_index=True)
    dftrades = dftrades.query("owner == @owner and expiration == @expiration")
    if not dftrades.empty:
      dftwideas = pd.concat([dftwideas,dftrades], ignore_index=True)

  ideatype = 'Idea'
  dfideas = pd.DataFrame(columns=['owner', 'active', 'quantity', 'Strike_Price', 'Action', 'price'])
  if 'project-ideas' in triggered[0] or 'apply-ideas' in triggered[0]:
    if 'apply-ideas' in triggered[0]:
      ideatype = 'Trade'
    filename = 'ideas_'+owner+'.csv' 
    dfideas = loadcsvtodf(ideas_directory, filename)
    if not dfideas.empty:
      dfideas = dfideas.query("owner == @owner")
      if not dfideas.empty:
       # yes, get trade start date which is needed as input
       help = dftwideas.query("transaction_date <= @quote_date and transaction_date > @lasttrade_close_date")[['owner','Strike_Price','trade_start_date','value_float','quantity_withsign']].groupby(['owner','Strike_Price','trade_start_date'],as_index=False).agg({"value_float": 'sum', "quantity_withsign": 'sum'})
       help = help.query("quantity_withsign != 0")
       if not help.empty:
        trade_start_date = help['trade_start_date'].values[0]
       else:
        trade_start_date = quote_date
       dfideas = adaptIdeas(dfideas, dftwideas[['Strike_Price','quantity_withsign']])
       dfideastrade = prepareTradeFromIdeas(dfideas, quote_date, expiration, trade_start_date, iv_ec, ideatype)
       if 'apply-ideas' in triggered[0]:
         savetocsvwithwaiting(dfideastrade, ideas_directory, filenameTrades, time_to_wait, 'a')
       dftwideas = pd.concat([dftwideas,dfideastrade], ignore_index=True)

  if not dftwideas.empty:
    dftwideas['pl_realized'] = dftwideas.apply(
      lambda row: getRealizedStrikePL(row["owner"], row["expiration"], row["Strike_Price"], row["Action"], row["Quantity"], row["value_float"], row["transaction_date"], row["trade_start_date"], dftw), axis=1
      )
      
  dftw_ow_ec_qd = dftwideas.query("transaction_date <= @quote_date and transaction_date > @lasttrade_close_date")[['owner','Strike_Price','trade_start_date','value_float','quantity_withsign']].groupby(['owner','Strike_Price','trade_start_date'],as_index=False).agg({"value_float": 'sum', "quantity_withsign": 'sum'})
  
  # Open price
  tw_status_pl_current, tw_status = getTradeStatusPL(dftw_ow_ec_qd, iv_ec)
  if tw_status.empty:
    isactivetrade = False
  else:
    isactivetrade = True

  if isactivetrade:
    # P&L current
    tw_status_pl_current = (tw_status_pl_current/1000).round(1) 
    trade_start_date = tw_status['trade_start_date'].values[0]
    # P&L in a day
    quote_date_previous = qdmarks[qdseq-1]
    if quote_date_previous < trade_start_date:
      tw_status_pl_previous = 0
    else:  
      iv_ec_previous = dfhist_rank.query("expiration == @expiration and quote_date == @quote_date_previous and delta >= @mindelta and delta <= @maxdelta")
      dftw_ow_ec_qd_previous = dftwideas.query("transaction_date <= @quote_date_previous and transaction_date > @lasttrade_close_date")[['owner','Strike_Price','trade_start_date','value_float','quantity_withsign']].groupby(['owner','Strike_Price','trade_start_date'],as_index=False).agg({"value_float": 'sum', "quantity_withsign": 'sum'})
      tw_status_pl_previous, tw_status_previous = getTradeStatusPL(dftw_ow_ec_qd_previous, iv_ec_previous)
      tw_status_pl_previous = (tw_status_pl_previous/1000).round(1) 
    tw_status_pl_day = (tw_status_pl_current - tw_status_pl_previous).round(1)
    # P&L expiration
    tw_status_pl_expiration = getTradePL(underlying_price, tw_status, dftw_ow_ec_qd, True)
    tw_status_pl_expiration = (tw_status_pl_expiration/1000).round(1)
    iv_ec['pl_expiration'] = iv_ec.apply(
        lambda row: getTradePL(row["strike"], tw_status, dftw_ow_ec_qd, True), axis=1) 
    # P&L theoretical
    iv_ec['pl_theoretical'] = iv_ec.apply(
        lambda row: getTradePL(row["strike"], tw_status, dftw_ow_ec_qd), axis=1) 
    # greeks trade current
    tw_status_delta = (100*tw_status['delta']*tw_status['quantity_withsign']).sum().round(0)
    tw_status_vega = (100*tw_status['vega']*tw_status['quantity_withsign']/1000).sum().round(1)
    # greeks trade over the whole expiration cycle 
    iv_ec['delta_trade_strike'] = iv_ec.apply(
        lambda row: getTradeGreek(row["strike"], tw_status, 'delta'), axis=1) 
    iv_ec['vega_trade_strike'] = iv_ec.apply(
        lambda row: getTradeGreek(row["strike"], tw_status, 'vega'), axis=1) 
    # PM (=portfolio margin) requirement theoretical
    naked_contracts = tw_status['quantity_withsign'].sum()
    total_contracts = tw_status.query("quantity_withsign > 0")['quantity_withsign'].sum()
    total_contracts = total_contracts - tw_status.query("quantity_withsign < 0")['quantity_withsign'].sum()
    tw_status_margin = getPortfolioMarginRequirement(underlying_price, iv_ec, naked_contracts, total_contracts)
    tw_status_margin = (tw_status_margin/1000).round(1)
    iv_ec['margin'] = iv_ec.apply(
        lambda row: getPortfolioMarginRequirement(row["strike"], iv_ec, naked_contracts, total_contracts), axis=1) 

  # P&L realized
  if dftwideas.empty:
    isplrealizedday = False
    tw_status_pl_realized_day = 0.0
  else:  
    tw_status_realized_day = dftwideas.query("transaction_date == @quote_date")[['trade_start_date','pl_realized']].groupby(['trade_start_date'],as_index=False).agg({"pl_realized": 'sum'})
    if tw_status_realized_day.empty:
      isplrealizedday = False
      tw_status_pl_realized_day = 0.0
    else:
      isplrealizedday = True
      tw_status_pl_realized_day = (tw_status_realized_day['pl_realized'].values[0]/1000).round(1)
      trade_start_date = tw_status_realized_day['trade_start_date'].values[0]

  if isplrealizedday or isactivetrade:
    tw_status_pl_realized_current = dftwideas.query("transaction_date <= @quote_date and transaction_date >= @trade_start_date")['pl_realized'].sum()
    tw_status_pl_realized_current = (tw_status_pl_realized_current/1000).round(1)
    din = (datetime.strptime(quote_date, '%Y-%m-%d') - datetime.strptime(trade_start_date, '%Y-%m-%d')).days

  # ------------------Trades List table-----------------------------
  dfTradesList = pd.DataFrame()
  if not dftwideas_allexp.empty:
   dfTradesList = dftwideas_allexp.query("Type == 'Trade'")[['trade_start_date','expiration']].groupby(['trade_start_date','expiration'],as_index=False).agg({})
   dfTradesList['Expiration'] = dfTradesList.apply(
       lambda row: row['expiration'][5:7]+'/'+row['expiration'][:4], axis = 1
       )
   dfTradesList = dfTradesList.drop(['expiration'], axis = 1)
   dfTradesList = dfTradesList.rename(columns={'trade_start_date': 'Start Date'})
  tableTradesList = dbc.Table.from_dataframe(dfTradesList, striped=True, hover=True, size='md')

  #------------------Trade Action Dates table-----------------------------
  dfTradeActionDates = pd.DataFrame()  #(columns=['owner', 'active', 'quantity', 'Strike_Price', 'Action', 'price'])
  if not dftwideas.empty and isactivetrade:
    dfTradeActionDates = dftwideas.query("transaction_date >= @trade_start_date and Type == 'Trade'")[['transaction_date']].groupby(['transaction_date'],as_index=False).agg({})
    dfTradeActionDates = dfTradeActionDates.rename(columns={'transaction_date': 'Transaction Date'})
  tableTradeActionDates = dbc.Table.from_dataframe(dfTradeActionDates, striped=True, hover=True, size='md')

  # ------------------Day Action table------------------------------
  dfDayActions = pd.DataFrame()
  if not dftwideas.empty:
    dfDayActions = dftwideas.query("transaction_date == @quote_date and Type == 'Trade'")[['Quantity','Strike_Price','Action','value_float']]
    dfDayActions["value_float"] = abs(dfDayActions["value_float"]/dfDayActions["Quantity"]/100).round(2)
    dfDayActions = dfDayActions.rename(columns={'Strike_Price': 'Strike', 'value_float': 'Price'})
  tableDayActions = dbc.Table.from_dataframe(dfDayActions, striped=True, hover=True, size='md')

  # ------------------TradeStatusOverview table------------------------------
  dfTradeStatusOverview = pd.DataFrame()
  if not dftwideas.empty:
    dftw_ow_ec_qd_noideas = dftwideas.query("transaction_date <= @quote_date and transaction_date > @lasttrade_close_date and Type != 'Idea'")[['owner','Strike_Price','trade_start_date','value_float','quantity_withsign']].groupby(['owner','Strike_Price','trade_start_date'],as_index=False).agg({"value_float": 'sum', "quantity_withsign": 'sum'}) 
    if not dftw_ow_ec_qd_noideas.empty and not dfideas.empty and ideatype == 'Idea':
      tw_status_pl_realized_current_noideas = dftwideas.query("transaction_date <= @quote_date and transaction_date >= @trade_start_date and Type != 'Idea'")['pl_realized'].sum()
      tw_status_pl_realized_current_noideas = (tw_status_pl_realized_current_noideas/1000).round(1)
      
      tw_status_realized_day_noideas = dftwideas.query("transaction_date == @quote_date and Type != 'Idea'")[['trade_start_date','pl_realized']].groupby(['trade_start_date'],as_index=False).agg({"pl_realized": 'sum'})
      if tw_status_realized_day_noideas.empty:
        tw_status_pl_realized_day_noideas = 0.0
      else:  
       tw_status_pl_realized_day_noideas = (tw_status_realized_day_noideas['pl_realized'].values[0]/1000).round(1)

      tw_status_pl_current_noideas, tw_status_noideas = getTradeStatusPL(dftw_ow_ec_qd_noideas, iv_ec)
      tw_status_pl_expiration_noideas = getTradePL(underlying_price, tw_status_noideas, dftw_ow_ec_qd_noideas, True)
      tw_status_pl_expiration_noideas = (tw_status_pl_expiration_noideas/1000).round(1)

      naked_contracts_noideas = tw_status_noideas['quantity_withsign'].sum()
      total_contracts_noideas = tw_status_noideas.query("quantity_withsign > 0")['quantity_withsign'].sum()
      total_contracts_noideas = total_contracts_noideas - tw_status_noideas.query("quantity_withsign < 0")['quantity_withsign'].sum()
      tw_status_margin_noideas = getPortfolioMarginRequirement(underlying_price, iv_ec, naked_contracts_noideas, total_contracts_noideas)
      tw_status_margin_noideas = (tw_status_margin_noideas/1000).round(1)

      tw_status_delta_noideas = (100*tw_status_noideas['delta']*tw_status_noideas['quantity_withsign']).sum().round(0)
      tw_status_vega_noideas = (100*tw_status_noideas['vega']*tw_status_noideas['quantity_withsign']/1000).sum().round(1)
      
      dfTradeStatusOverview = pd.DataFrame({
          "": ["w/o", "incl."], 
          "P/L realized": [str(tw_status_pl_realized_current_noideas)+"K", str(tw_status_pl_realized_current)+"K"],  
          "P/L realized Day": [str(tw_status_pl_realized_day_noideas)+"K", str(tw_status_pl_realized_day)+"K"],
          "P/L expiration": [str(tw_status_pl_expiration_noideas)+"K", str(tw_status_pl_expiration)+"K"],
          "Portfolio Margin": [str(tw_status_margin_noideas)+"K", str(tw_status_margin)+"K"],
          "DELTA": [str(tw_status_delta_noideas), str(tw_status_delta)],
          "VEGA": [str(tw_status_vega_noideas)+"K", str(tw_status_vega)+"K"],
      })
    elif not dftw_ow_ec_qd_noideas.empty or not dfideas.empty:
      dfTradeStatusOverview = pd.DataFrame({
          "P/L realized": [str(tw_status_pl_realized_current)+"K"],
          "P/L realized Day": [str(tw_status_pl_realized_day)+"K"],
          "P/L expiration": [str(tw_status_pl_expiration)+"K"],
          "Portfolio Margin": [str(tw_status_margin)+"K"],
          "DELTA": [str(tw_status_delta)],
          "VEGA": [str(tw_status_vega)+"K"],
      })
  tableTradeStatusOverview = dbc.Table.from_dataframe(dfTradeStatusOverview, striped=True, bordered=True, hover=True, size='md')

  # ------------------Ideas Graph------------------------------
  
  # underlying history + current price if needed
  df_coupon = dfhist_rank.query("expiration == @expiration and dte >= @dte and dte<=@dte+@lookbackperiod_ideas")[['quote_date','dte','underlying_price']].groupby(['quote_date','dte'],as_index=False).agg({"underlying_price": 'min'})
  if not quote_date in df_coupon['quote_date'].values:
    df_coupon.loc[len(df_coupon.index)] = [quote_date, dte, underlying_price]
  df_coupon = df_coupon.assign(dtp = (pd.to_datetime(df_coupon['quote_date']).astype('datetime64[ns]')-datetime.strptime(quote_date, '%Y-%m-%d')).dt.days)
  df_coupon = df_coupon.assign(coupon = 0.0)

  if not dfideas.empty and ideatype == 'Idea':
    quote_date_min = df_coupon['quote_date'].min()
    iv_ec_min = dfhist_rank.query("expiration == @expiration and quote_date == @quote_date_min")
    dfideas4coupon = prepareTradeFromIdeas(dfideas, quote_date_min, expiration, quote_date_min, iv_ec_min, 'Idea')
    # coupon projections
    dfhist_rank_ec = dfhist_rank.query("expiration == @expiration")
    df_coupon['coupon'] = df_coupon.apply(
       lambda row: (-1)*getIdeaStatusPL(owner, expiration, row['quote_date'], quote_date_min, dfideas4coupon, iv_ec, dfhist_rank_ec), axis=1) 
    #define color for coupon line
    npcoupon = df_coupon["coupon"].to_numpy() 
    wsdelta = 0
    for i, value in enumerate(npcoupon):
      if i>0:
       delta = npcoupon[i]-npcoupon[i-1]
       wsdelta = wsdelta + delta*i  
    color_coupon = get_continuous_color(colorscale_bluered, intermed=ideacoloraccelerator*wsdelta/abs(dfideas4coupon['value_float'].sum()))

    df_coupon = df_coupon.assign(deltatime = '')
    df_coupon.loc[df_coupon["dtp"]>=-60, 'deltatime'] = '-2M'
    df_coupon.loc[df_coupon["dtp"]>=-30, 'deltatime'] = '-1M'
    df_coupon.loc[df_coupon["dtp"]>=-7, 'deltatime'] = '-1W'
    previous_trading_day = df_coupon.query("dtp<0")['dtp'].max()
    df_coupon.loc[df_coupon["dtp"]>=previous_trading_day, 'deltatime'] = '-1D'
    df_coupon_deltatime = df_coupon[['quote_date','coupon','deltatime']].groupby(['deltatime'],as_index=False).agg({"quote_date": 'min'})
    df_coupon_deltatime = pd.merge(df_coupon_deltatime,df_coupon, how='left',left_on=['quote_date','deltatime'],right_on=['quote_date','deltatime'])
  
  fig_ideas = make_subplots(specs=[[{"secondary_y": True}]])
 
  fig_ideas.add_trace(go.Scatter(
      x=df_coupon["quote_date"],
      y=df_coupon["underlying_price"],
      # hoverinfo='skip',
      opacity=0.4,
      name="Underlying",
      marker_color='yellow'),
      secondary_y=True,
  )
  if not dfideas.empty and ideatype == 'Idea':
    fig_ideas.add_trace(go.Scatter(
       x=df_coupon["quote_date"],
       y=df_coupon["coupon"],
       customdata = np.stack((df_coupon['dtp'], df_coupon['coupon']/1000), axis=-1),
       hovertemplate=
       '<b>%{customdata[1]:.1f</b>}K<br>' + 
       't%{customdata[0]:.0f}<br>' +
       '<extra></extra>',
       line_width = 2,
       opacity=1,
       name="Coupon",
       marker_color=color_coupon),
       secondary_y=False,
    )
    fig_ideas.add_trace(go.Scatter(
        x=df_coupon_deltatime["quote_date"],
        y=df_coupon_deltatime["coupon"],
        hoverinfo='skip',
        mode='markers+text',
        text=df_coupon_deltatime["deltatime"],
        textposition="top center",
        textfont = dict(
            color='yellow',
            size = 13,
        ),
        marker = dict(
          symbol = 'x',
          color='yellow',
          opacity = 1,
          size = 10,
        ),
      ),
      secondary_y=False,
    )

    # Set y-axis
    fig_ideas.update_yaxes(
        title_text="<b>Coupon</b>",
        secondary_y=False,
        showgrid=False,
        # gridwidth=0.1,
        # gridcolor="gray",
        # zeroline=False,
        layer='below traces',
    )

  # Set x-axis
  fig_ideas.update_xaxes(
      showspikes=True, # Show spike line for X-axis
      # Format spike
      spikethickness=2,
      spikedash="dash",
      spikecolor="white",
      spikemode="across",
      zeroline=False,
      showgrid=False,
      layer='below traces',
      # showticklabels=False,
  )

  fig_ideas.update_yaxes(
      title_text="<b>Underlying</b>",
      secondary_y=True,
      showgrid=False,
      zeroline=False,
      layer='below traces',
    )
  
  fig_ideas.update_layout(
    # height=500,
    # width=300,
    # title='Coupon',
    # xaxis_title="time",
    plot_bgcolor='#111111', #'rgb(232,250,250)',
    paper_bgcolor='#111111',
    font_color='#7FDBFF',
    showlegend=False,
    hovermode="x",
    # hoverlabel=dict(
    #     bgcolor="yellow",
    #     font_size=14,
    #     #font_family="Rockwell"
    # )
  ) 


  # ------------------IV Graph------------------------------

  tw_yaxis='ivrank_1M'
  if '1M' not in checklistrank:
    if '1W' in checklistrank:
      tw_yaxis='ivrank_1W'
    else:
      if '2M' in checklistrank:
        tw_yaxis='ivrank_2M'  

  # expiration cycle IVR color
  ivrank_ec_1M = iv_ec_color['ivrank_ec_1M'].values[0]
  color_ec_1M = get_continuous_color(colorscale_bluered, intermed=ivrank_ec_1M/100)  # 1-ivrank_ec_1M/100 for colors switch
  ivrank_ec_2M = iv_ec_color['ivrank_ec_2M'].values[0]
  color_ec_2M = get_continuous_color(colorscale_bluered, intermed=ivrank_ec_2M/100)
  ivrank_ec_1W = iv_ec_color['ivrank_ec_1W'].values[0]
  color_ec_1W = get_continuous_color(colorscale_bluered, intermed=ivrank_ec_1W/100)
  fig = make_subplots(specs=[[{"secondary_y": True}]])

  # Open interest & volume
  if 'OI' in checklistiv:
    fig.add_trace(go.Bar(
        x=iv_ec["strike"],
        y=iv_ec["openinterest"],
        hoverinfo='skip',
        opacity=0.2,
        width=30,
        name="Open Interest",
        marker_color='blue'),
        secondary_y=True,
    )
  if 'VOL' in checklistiv:
    fig.add_trace(go.Bar(
        x=iv_ec["strike"],
        y=iv_ec["volume"],
        hoverinfo='skip',
        opacity=0.2,
        width=30,
        name="Volume",
        marker_color='aquamarine'), #aliceblue, antiquewhite, aqua, aquamarine, azure,
        secondary_y=True,
    )
      
  # RANKS
  if '1M' in checklistrank:
    if isactivetrade:
      fig.add_trace(go.Scatter(
          x=iv_ec["strike"],
          y=iv_ec["ivrank_1M"].round(0),
          # text = iv_ec["delta"].round(2),
          customdata = np.stack((iv_ec['delta'], iv_ec['pl_theoretical']/1000, iv_ec['pl_expiration']/1000, iv_ec['delta_trade_strike'], iv_ec['vega_trade_strike']/1000, iv_ec['margin']/1000, iv_ec['mid']), axis=-1),
          hovertemplate=
          '<b>%{y:.0f}</b> IV Rank 1M<br>' +
          # 'delta: %{text}<br>'+
          'delta: %{customdata[0]:.2f}<br>' +
          'mid: %{customdata[6]:.1f}<br>' +
          '---------theo-------<br>' +
          'delta: %{customdata[3]:.0f}<br>' +
          'vega:  %{customdata[4]:.1f}K<br>' +
          'PM:  %{customdata[5]:.1f}K<br>' +
          'P/L:   %{customdata[1]:.1f}K<br>' +
          '---------exp-------<br>' +
          'P/L:   %{customdata[2]:.1f}K<br>' +
          '<extra></extra>',
          name="IV Rank 1M",
          mode='lines',
          line_width = 4,
          marker_color=color_ec_1M,
          opacity=1),
          secondary_y=False,
      )
    else:
      fig.add_trace(go.Scatter(
          x=iv_ec["strike"],
          y=iv_ec["ivrank_1M"].round(0),
          customdata = np.stack((iv_ec['delta'], iv_ec['mid']), axis=-1),
          hovertemplate=
          'IVR1M: <b>%{y:.0f}</b><br>' +
          'delta: %{customdata[0]:.2f}<br>' +
          'mid: %{customdata[1]:.1f}<br>' +
          '<extra></extra>',
          name="IV Rank 1M",
          mode='lines',
          line_width = 4,
          marker_color=color_ec_1M,
          opacity=1),
          secondary_y=False,
      )
  if '1W' in checklistrank:
    fig.add_trace(go.Scatter(
        x=iv_ec["strike"],
        y=iv_ec["ivrank_1W"].round(0),
        name="IV Rank 1W",
        mode='lines',
        line_width = 2,
        marker_color=color_ec_1W,
        opacity=1),
        secondary_y=False,
    )
  if '2M' in checklistrank:
    fig.add_trace(go.Scatter(
        x=iv_ec["strike"],
        y=iv_ec["ivrank_2M"].round(0),
        name="IV Rank 2M",
        mode='lines',
        line_width = 1,
        marker_color=color_ec_2M,
        opacity=1),
        secondary_y=False,
    )

  if 'IVP' in checklistiv:
    if '1M' in checklistrank:
      fig.add_trace(go.Scatter(
          x=iv_ec["strike"],
          y=iv_ec["ivpercentile_1M"].round(0),
          name="IV Percentile 1M",
          mode='lines',
          line_width = 4,
          marker_color='white',
          opacity=1),
          secondary_y=False,
      )
    if '1W' in checklistrank:
      fig.add_trace(go.Scatter(
          x=iv_ec["strike"],
          y=iv_ec["ivpercentile_1W"].round(0),
          name="IV Percentile 1W",
          mode='lines',
          line_width = 2,
          marker_color='white',
          opacity=0.66),
          secondary_y=False,
      )
    if '2M' in checklistrank:
      fig.add_trace(go.Scatter(
          x=iv_ec["strike"],
          y=iv_ec["ivpercentile_2M"].round(0),
          name="IV Percentile 2M",
          mode='lines',
          line_width = 1,
          marker_color='white',
          opacity=0.33),
          secondary_y=False,
      )

  # Trade status  
  if '1M' in checklistrank or '1W' in checklistrank or '2M' in checklistrank:
    fig.add_trace(go.Scatter(
        x=tw_status["strike"],
        y=tw_status[tw_yaxis],
        hoverinfo='skip',
        mode='markers+text',
        text=tw_status["quantity_withsign"],
        textposition="middle center",
        textfont = dict(
            color='yellow',
        ),
        marker = dict(
            symbol = 'hexagon',
            color=list(map(SetColorTWStatus, tw_status["quantity_withsign"])),
            opacity = 1,
            size = 30,
            line = dict(
                color = 'yellow',
                width = 1,
            )
        ),
        opacity = 1,),
        secondary_y=False,
    )
    # Day actions 
    tw_action_ec_qd = dftwideas.query("owner == @owner and expiration == @expiration and transaction_date == @quote_date")[['Strike_Price','value_float','quantity_withsign','quantity_withsign_realized']].groupby(['Strike_Price'],as_index=False).agg({"value_float": 'sum', "quantity_withsign": 'sum', "quantity_withsign_realized": 'sum'})
    tw_action = iv_ec.iloc[np.where(iv_ec.strike.isin(np.sort(tw_action_ec_qd["Strike_Price"].unique())))]
    tw_action = pd.merge(tw_action,tw_action_ec_qd, how='left',left_on=['strike'],right_on=['Strike_Price']) 
    if dayaction == 'realized' or dayaction == 'all':
      if dayaction == 'realized':
        action_quantity_colname = 'quantity_withsign_realized'
        tw_action = tw_action.query("quantity_withsign_realized != 0")
      else:
        action_quantity_colname = 'quantity_withsign'
      fig.add_trace(go.Scatter(
          x=tw_action["strike"],
          y=tw_action[tw_yaxis],
          hoverinfo='skip',
          mode='markers+text',
          text=tw_action[action_quantity_colname],
          textposition="bottom center",
          textfont = dict(
              color=list(map(SetColorTWStatus, tw_action[action_quantity_colname])),
              size = 18,
          ),
          marker = dict(
            symbol = 'circle-open',
            color=list(map(SetColorTWStatus, tw_action[action_quantity_colname])),
            opacity = 1,
            size = 35,
            line = dict(
                width = 3,
            )
          ),
        ),
        secondary_y=False,
    )

  # Underlying price
  fig.add_shape(
      type="line",
      yref="paper",
      x0=underlying_price, y0=0, x1=underlying_price, y1=1,
      opacity=0.6,
      line=dict(
          color="yellow",
          width=2,
          #dash="dot",
          ),
  )

  # Inside short
  insideshortbottom = iv_ec.query("delta <= @insideshortdeltafrom")['strike'].min()
  insideshortup = iv_ec.query("delta >= @insideshortdeltato")['strike'].max()
  fig.add_shape(
      type="rect",
      x0=insideshortbottom, y0=-100, x1=insideshortup, y1=200,
      fillcolor="Red",
      opacity = 0.15,
  )

  # Long
  longbottom = insideshortup
  longup = iv_ec.query("delta >= @longdeltato")['strike'].max()
  fig.add_shape(
      type="rect",
      x0=longbottom, y0=-100, x1=longup, y1=200,
      fillcolor="Green",
      opacity = 0.15,
  )

  # Outside short
  outsideshortbottom = iv_ec.query("delta <= @outsideshortdeltafrom")['strike'].min()
  outsideshortup = iv_ec.query("delta >= @outsideshortdeltato")['strike'].max()
  fig.add_shape(
      type="rect",
      x0=outsideshortbottom, y0=-100, x1=outsideshortup, y1=200,
      fillcolor="Red",
      opacity = 0.15,
  )

  # Set x-axis
  fig.update_xaxes(
      showspikes=True, # Show spike line for X-axis
      # Format spike
      spikethickness=2,
      spikedash="dash",
      spikecolor="white",
      spikemode="across",
      zeroline=False,
      showgrid=False,
      layer='below traces',
  )

  # Set y-axis
  fig.update_yaxes(
      title_text="<b>RANK</b>",
      secondary_y=False,
      gridwidth=0.1,
      gridcolor="gray",
      # zeroline=False,
      layer='below traces',
      range= [iv_ec["ivrank_1M"].min() - min(20,0.5 * abs(iv_ec["ivrank_1M"].min())), iv_ec["ivrank_1M"].max() + min(20,1.5 * abs(iv_ec["ivrank_1M"].max()))]
  )
  if 'OI' in checklistiv or 'VOL' in checklistiv:
    fig.update_yaxes(
        title_text="<b>PIECES</b>",
        secondary_y=True,
        showgrid=False,
        zeroline=False,
        layer='below traces',
    )

  if isactivetrade:
    titletxt = "in "+str(din)+" | "+"<b>"+str(dte)+"</b> dte ... P/L <b>"+str(tw_status_pl_current)+"K</b>                                P/L Day: <b>"+str(tw_status_pl_day)+"K</b>"
  else:
    if isplrealizedday:
      titletxt = "in "+str(din)+" | "+str(dte)+" dte ... P/L <b>"+str(tw_status_pl_realized_current)+"K</b>                                      Trade <b>CLOSED</b>"
    else:  
      titletxt = "<b>"+str(dte)+"</b> dte"  
  # Update layout
  fig.update_layout(
      # height=400, width=1390,
      title=titletxt,
      barmode="stack",
      plot_bgcolor='#111111', #'rgb(232,250,250)',
      paper_bgcolor='#111111',
      font_color='#7FDBFF',
      showlegend=False,
      hovermode="x",
      # hoverlabel=dict(
      #     bgcolor="yellow",
      #     font_size=14,
      #     #font_family="Rockwell"
      # )
  ) 

  return [tableTradeActionDates, tableTradesList, tableDayActions, quote_date, qdseqreturn, expdropopt, expiration, expseqreturn, expslidermarks, tableTradeStatusOverview, fig_ideas, fig]

# @app.callback(
#     Output('check-output', 'children'),
#     Input("tabs-owner", "active_tab"),
#     Input({'type': 'checklist-active', 'index': ALL}, 'value'),
#     Input({'type': 'input-quantity', 'index': ALL}, 'value'),
#     Input({'type': 'input-strike', 'index': ALL}, 'value'),
#     Input({'type': 'dropdown-action', 'index': ALL}, 'value'),
#     Input({'type': 'button-remove', 'index': ALL}, 'n_clicks'),
#     )
# def display_output(owner, active, quantity, strike, action, remove):
#     return html.Div([
#                     html.Div('active Input = {}'.format(active[0])),
#                     html.Div('quantity Input = {}'.format(quantity[0])),
#                     html.Div('strike Input = {}'.format(strike[0])),
#                     html.Div('action Input = {}'.format(action[0])), 
#                     html.Div('remove Input = {}'.format(remove[0])),
#         # html.Div('Strike Input {} = {}'.format(i + 1, value)) for (i, value) in enumerate(strike)
#     ])

if __name__ == "__main__":
  app.run_server(mode='inline', debug=True, use_reloader=False)